In [4]:
import requests
import json
import time 
import pandas as pd
from bs4 import BeautifulSoup

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from requests.exceptions import RequestException
import re
import pandas as pd

def get_one_page(url):
    headers = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3135.4 Safari/537.36"
    }
    try:
        response=requests.get(url,headers=headers)
        if(response.status_code==200):
            response.encoding='utf-8'
            return response.text
        return None
    except RequestException:
        print("error")
        return None 
    
def parse_one_page(html):
    soup=BeautifulSoup(html,'lxml')
    comment_lists=soup.find_all('div','comment')
    comment_lists


    #获取一个页面的用户名
    user_ls='p class="sub">.*?<span class=".*?">.*?<em></em>.*?</span>.*?<span>(.*?)</span>.*?'
    #user_ls = 'p class="sub">.*?<span class=".*?">.*?</span>.*?<span>(.*?)</span>.*?'
    user_pattern = re.compile(user_ls, re.S)
    user_lists = re.findall(user_pattern, str(comment_lists))
    for li in user_lists:
        #print(li)
        res_users.append(li)

    #获取一个页面的评分
    fen_ls = 'p class="sub">.*?<span class="(.*?)">.*?'
    fen_pattern = re.compile(fen_ls, re.S)
    fen_lists = re.findall(fen_pattern, str(comment_lists))
    for li in fen_lists:
        #print(li)
        res_pingfen.append(li)
    #获取一个页面的时间
    shijians=soup.select('.comment .frt')
    
    for li in shijians:
        #print(li.get_text())
        res_times.append(li.get_text())
    #获取一个页面的评论
    pingluns=soup.select('.comment .content')
    for li in pingluns:
        #print(li.get_text())
        res_comtents.append(li.get_text())
        
def pachong(offset):
    url = 'http://app.hicloud.com/comment/commentAction.action?&appId=C10465259&appName=%E5%B9%B3%E5%AE%89%E5%8F%A3%E8%A2%8B%E9%93%B6%E8%A1%8C&rating=0&_page='+str(offset)
    html = get_one_page(url)
    parse_one_page(html)

def Huawei_application_market(file_path):
    start_time = time.time() 
    pool = ThreadPool(5)
    pool.map(pachong,[i for i in range(1,87)])
    pool.close()
    pool.join()
    data=[res_users,res_pingfen,res_times,res_comtents]
    result=pd.DataFrame(data)
    result=result.T
    result.info()
    result.columns=['用户名称','评分','时间','评论']
    result.to_csv(file_path+'华为_平安口袋银行.csv')
    print("爬取成功！")
    
    
if __name__ == '__main__':
    res_users=[]
    res_pingfen=[]
    res_times=[]
    res_comtents=[]
    file_path='C:\\Users\\pujing\\Desktop\\长亮科技\\'
    Huawei_application_market(file_path)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 4 columns):
0    440 non-null object
1    440 non-null object
2    440 non-null object
3    440 non-null object
dtypes: object(4)
memory usage: 13.8+ KB
爬取成功！
